In [6]:
import pandas as pd
from database import select_dataset
import numpy as np

In [7]:
# FIND WAY TO WRAP THIS CLEANLY CAUSE IT's A MESS !

aquastat = select_dataset('aquastat').dropna()
evapotranspiration = select_dataset('evapotranspiration').dropna().drop_duplicates()
cropcalendar = select_dataset('cropcalendar').drop(columns=['Year'])
efficientwater = select_dataset('efficientwater')
wb_data = select_dataset('worldbank')

In [8]:
def pivot(df):
    return df.pivot(index=['ISO', 'Year'], columns='Variable', values='Value')

def fill_missing_values(df):
    return df.groupby(level='ISO').fillna(method='ffill')\
             .groupby(level='ISO').fillna(method='bfill')

def df_to_dict(df):
    X = {}
    for code in df.columns:
        X[code] = df[code].fillna(0)
    return X

In [9]:
def process_datasets(aquastat, efficientwater, wb_data,  evapotranspiration, cropcalendar):
    '''To cleanup !!! '''
    
    aquastat = aquastat[~aquastat.ISO.isin(['USA', 'CHN', 'IND', 'RUS'])] # Not parsed !
    aquastat = aquastat[aquastat.Variable != 'CI']
    aquastat['Year'] = aquastat['Year'].astype(int)
    wb_data['Year'] = wb_data['Year'].astype(int)
    
    aquastat = fill_missing_values(pivot(aquastat))
    evapotranspiration = pivot(evapotranspiration)
    ETa = fill_missing_values(evapotranspiration['ETa'])
    ETo = evapotranspiration['ETo'].reset_index('Year', drop=True).dropna()
    cropcalendar = cropcalendar.pivot(index=['ISO', 'Item'], columns='Variable', values='Value')
    wb_data = pivot(wb_data)
    efficientwater = pivot(efficientwater)

    X = df_to_dict(aquastat)
    
    gva = df_to_dict(wb_data)
    ew = df_to_dict(efficientwater)
    
    X.update(gva)
    X.update(ew)
    
    X['ETa'] = ETa
    X['ETo'] = ETo
    X['LWU'] = 0
    X['KC'] = cropcalendar['Kc']
    X['ICA'] = cropcalendar['ICA']
    
    
    return X

In [10]:
data_dict = process_datasets(aquastat, efficientwater, wb_data,  evapotranspiration, cropcalendar)

# Model

In [11]:
from graphmodels.graphmodel import GraphModel, converte_to_format, concatenate_graph_specs
import numpy as np
from graphmodels.validation import score_model
from models.Sarah.model_EW import EW_models, MWU_model
from graphmodels.utils import get_X_y_from_data

In [12]:
for model_name, model in EW_models.items():
    print(model_name, ': ')
    X, y = get_X_y_from_data(model, data_dict)
    print(score_model(model, X, y))

IWW_model : 
         r2  correlation         rmse Variable
0 -0.126651     0.937999   179.193726      IWR
1 -4.205210     0.418487  1603.099040      IWW
2 -0.062886     0.913503   575.215642      AWU
EW1_partial_model : 
         r2  correlation         rmse Variable
0  0.989945     0.994980     5.560493      TWW
1 -1.318223     0.491137  8018.101538      EW1
EW2_partial_model : 
         r2  correlation          rmse Variable
0  0.976665     0.990610  13890.813076      TFA
1 -0.111783     0.251153  54394.503560      EW2
EW1_model : 


/home/simon/anaconda3/lib/python3.7/site-packages/pandas/core/arraylike.py:358: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


         r2  correlation         rmse Variable
0 -0.126651     0.937999   179.193726      IWR
1 -4.205210     0.418487  1603.099040      IWW
2 -0.062886     0.913503   575.215642      AWU
3  0.885780     0.945409     0.928600      MWU
4  0.185689     0.904184   648.592989      TWW
5  0.371841     0.670514  2356.053564      EW1
EW2_model : 
         r2  correlation          rmse Variable
0 -0.126651     0.937999    179.193726      IWR
1 -4.205210     0.418487   1603.099040      IWW
2 -0.062886     0.913503    575.215642      AWU
3  0.885780     0.945409      0.928600      MWU
4  0.976665     0.990610  13890.813076      TFA
5 -0.111783     0.251153  54394.503560      EW2


/home/simon/anaconda3/lib/python3.7/site-packages/pandas/core/arraylike.py:358: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


# Test on model

In [ ]:
WP = data_dict['WP']
GDPC = data_dict['GDPC']
Pop = data_dict['Pop']
MWU = data_dict['MWU']

scenario_df = pd.concat([WP, GDPC, Pop, MWU], axis=1).rename(columns={0: 'GDPC'}).reset_index()
ISO_scenario_df = scenario_df.query("ISO == 'FRA' and Year > 2005")\
                             .fillna(method='ffill')\
                             .fillna(method='bfill')

X = df_to_dict(ISO_scenario_df.set_index(['ISO', 'Year']))

In [ ]:
MWU_comp = MWU_model.run(X)['MWU']

In [ ]:
ISO_scenario_df['MWU_comp'] = MWU_comp.values

In [ ]:
ISO_scenario_df

In [ ]:
MWU_model.draw()

In [ ]:
aquastat.query("Variable == 'MWU'")